# チュートリアルの使い方

このチュートリアルでは、ワークショップ内のすべてのノートブックの要件と環境変数が保存されるように、最初にこの初期ノートブックを実行する必要があります。

## 始め方

1. インストラクター主導のワークショップに参加している場合、または提供された [CloudFormation Template](https://raw.githubusercontent.com/aws-samples/prompt-engineering-with-anthropic-claude-v-3/main/cloudformation/workshop-v1-final-cfn.yml) を使用してワークショップインフラストラクチャを展開した場合は、ステップ2に進むことができます。そうでない場合は、ワークショップの [GitHub Repository](https://github.com/aws-samples/prompt-engineering-with-anthropic-claude-v-3) をローカルマシンにダウンロードする必要があります。

2. 次のコマンドを実行して、必要な依存関係をインストールします：

> ⚠️ **pipの依存関係解決に関連するエラーメッセージは無視してください。**

In [ ]:
%pip install -qU pip
%pip install -qUr requirements.txt --force-reinstall

ノートブックのセルを順番に実行し、提供された指示に従ってください。

---

## 使用上の注意とヒント 💡

- このコースでは、温度0のClaude 3 Haikuを使用します。温度についてはコースの後半で詳しく説明しますが、今のところ、これらの設定がより決定論的な結果をもたらすことを理解しておけば十分です。このコースのすべてのプロンプトエンジニアリング技術は、Claude 2やClaude Instant 1.2などの以前の世代のレガシーClaudeモデルにも適用されます。

- `Shift + Enter`を使用してセルを実行し、次のセルに移動できます。

- チュートリアルページの下部に到達したら、フォルダ内の次の番号付きファイルに移動するか、その章のファイル内のコンテンツが終了した場合は次の番号付きフォルダに移動してください。

### Boto3 SDKとConverse API
このチュートリアルでは、[Amazon Boto3 SDK](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-runtime.html)と[Converse API](https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/bedrock-runtime/client/converse.html)を使用します。

以下は、このチュートリアルでプロンプトを実行する際の例です。まず、`get_completion`を作成します。これは、プロンプトをClaudeに送信し、Claudeが生成した応答を返すヘルパー関数です。そのセルを今すぐ実行してください。

まず、モデル名とリージョンを設定して保存します。

In [ ]:
import boto3
import json
from datetime import datetime
from botocore.exceptions import ClientError

session = boto3.Session()
region = session.region_name

In [ ]:
#modelId = 'anthropic.claude-3-sonnet-20240229-v1:0'
modelId = 'anthropic.claude-3-haiku-20240307-v1:0'

%store modelId
%store region

print(f'使用中のmodelId: {modelId}')
print('使用中のregion: ', region)

bedrock_client = boto3.client(service_name = 'bedrock-runtime', region_name = region,)

次に、`get_completion`を作成します。これは、プロンプトをClaudeに送信し、Claudeが生成した応答を返すヘルパー関数です。今、そのセルを実行してください。

In [ ]:
def get_completion(prompt, system_prompt=None):
    # 推論設定を定義します
    inference_config = {
        "temperature": 0.0,  # 多様な応答を生成するための温度を設定します
        "maxTokens": 200  # 生成するトークンの最大数を設定します
    }
    # 追加のモデルフィールドを定義します
    additional_model_fields = {
        "top_p": 1,  # 核サンプリングのためのtop_p値を設定します
    }
    # 会話メソッドのパラメータを作成します
    converse_api_params = {
        "modelId": modelId,  # 使用するモデルIDを指定します
        "messages": [{"role": "user", "content": [{"text": prompt}]}],  # ユーザーのプロンプトを提供します
        "inferenceConfig": inference_config,  # 推論設定を渡します
        "additionalModelRequestFields": additional_model_fields  # 追加のモデルフィールドを渡します
    }
    # system_textが提供されているか確認します
    if system_prompt:
        # system_textが提供されている場合、converse_params辞書にシステムパラメータを追加します
        converse_api_params["system"] = [{"text": system_prompt}]

    # Bedrockクライアントにリクエストを送信して応答を生成します
    try:
        response = bedrock_client.converse(**converse_api_params)

        # 応答から生成されたテキストコンテンツを抽出します
        text_content = response['output']['message']['content'][0]['text']

        # 生成されたテキストコンテンツを返します
        return text_content

    except ClientError as err:
        message = err.response['Error']['Message']
        print(f"クライアントエラーが発生しました: {message}")

今からClaudeのためのプロンプトの例を書き出し、`get_completion`ヘルパー関数を実行してClaudeの出力を表示します。以下のセルを実行すると、その下にClaudeからの応答が表示されます。

プロンプト文字列を自由に変更して、Claudeから異なる応答を引き出してみてください。

In [ ]:
# プロンプト
prompt = "Hello, Claude!"

# Claudeの応答を取得
print(get_completion(prompt))

以前に定義された`modelId`と`region`変数は、チュートリアル全体で使用されます。各チュートリアルページのセルを上から下へ実行することを確認してください。